In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

C:\Programming\anaconda3-4.4.0\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [172]:
# Download the latest SO dataset here: https://drive.google.com/open?id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV
data_2019 = pd.read_csv("developer_survey_2019/survey_results_public.csv", low_memory=False)

In [173]:
to_drop = ["Respondent", "OpenSource", "CareerSat", "JobSat", "JobSeek", "ResumeUpdate", "SurveyLength", "SurveyEase", "WelcomeChange", "EntTeams", "ScreenName", "LastIn", "SO", "Blockchain", "WorkChallenge", "BetterLife", "OffOn", "Currency", "CompTotal", "CompFreq", "MainBranch", "ITPerson"]

In [174]:
def col_drop(df, to_drop):

    df_dropped = df.copy()

    for flag in to_drop:
        try:
            df_dropped.drop([x for x in df_dropped.columns if flag in x], axis=1, inplace=True)
        except:
            pass

    return df_dropped

In [175]:
data_2019 = col_drop(data_2019, to_drop)
print("Success!")
print(data_2019.head().T.shape)  

Success!
(51, 5)


In [176]:
# Only consider those with income between $10,000 and $250,000
data_2019 = data_2019[(data_2019["ConvertedComp"] >= 10000) & (data_2019["ConvertedComp"] <= 250000)]
data_2019["ConvertedComp"] = np.log(data_2019["ConvertedComp"])
data_2019 = data_2019.rename(columns = {"ConvertedComp": "Income"})

# Only consider US respondents
data_2019 = data_2019[data_2019["Country"] == "United States"]

# Only consider 18+ respondents
data_2019 = data_2019[data_2019["Age"] >= 18]

# Only consider respondents in the workforce
data_2019 = data_2019[data_2019["Employment"] != "Retired"]
data_2019 = data_2019[data_2019["Employment"] != "Not employed, and not looking for work"]

# Only consider those with at least some education
data_2019 = data_2019[data_2019["EdLevel"] != "I never completed any formal education"]

data_2019 = data_2019.fillna("no_answer")

# Convert numeric columns to int
data_2019["Age"] = data_2019["Age"].astype("int32", errors="ignore")
data_2019["Age1stCode"] = data_2019["Age1stCode"].astype("int32", errors="ignore")
data_2019["YearsCode"] = data_2019["YearsCode"].astype("int32", errors="ignore")
data_2019["WorkWeekHrs"] = data_2019["WorkWeekHrs"].astype("int32", errors="ignore")
data_2019["CodeRevHrs"] = data_2019["CodeRevHrs"].astype("int32", errors="ignore")

# Exclude respondents who selected multiple gender, race, or sexual orientation
# options
data_2019 = data_2019[~data_2019["Gender"].str.contains(";")]
data_2019 = data_2019[~data_2019["Ethnicity"].str.contains(";")]
data_2019 = data_2019[~data_2019["Sexuality"].str.contains(";")]

# Reset index
data_2019 = data_2019.reset_index(drop=True)

In [225]:
# TODO: Bucketing for numeric responses (Age, Age1stCode, YearsCode, WorkWeekHrs, CodeRevHrs)

age_labels = ["-20", "21-25", "26-30", "31-35", "35-40", "41-45", "45-50", "51-55", "55-60", "61-65", "65-"]
year_buckets = ["0-5", "6-10", "11-15", "16-20", "21-25", "26-30", "31-35", "35-40", "40-"]
work_week_hour_buckets = ["0-5", "6-10", "11-15", "16-20", "21-25", "26-30", "31-35", "35-40", "41-45", "46-50", "51-55", "56-60", "61-65", "66-70", "71-75", "76-80"]
code_rev_hour_buckets = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11-15", "16-20", "21-25", "26-30", "30-"]

age_buckets = np.array([0,20,25,30,35,40,45,50,55,60,65,100])
year_buckets = np.array([0,5,10,15,20,25,30,35,40,100])

In [226]:
a = pd.cut(data_2019["Age"], age_buckets, labels=age_labels)
#y = pd.cut(data_2019["YearsCode"], year_buckets)

In [228]:
a

0        21-25
1        26-30
2        45-50
3        21-25
4        31-35
5        31-35
6        31-35
7        41-45
8        61-65
9        41-45
10         -20
11       26-30
12       31-35
13       41-45
14       41-45
15       26-30
16       21-25
17       31-35
18       61-65
19       26-30
20       35-40
21       31-35
22       35-40
23       31-35
24       26-30
25       21-25
26       21-25
27       26-30
28       41-45
29       31-35
         ...  
11398    31-35
11399    41-45
11400    21-25
11401    45-50
11402    41-45
11403    26-30
11404    21-25
11405    31-35
11406    26-30
11407    26-30
11408    26-30
11409    26-30
11410    26-30
11411    51-55
11412    26-30
11413    21-25
11414    35-40
11415    45-50
11416    26-30
11417    26-30
11418    41-45
11419    35-40
11420    26-30
11421    26-30
11422    31-35
11423    26-30
11424    35-40
11425    35-40
11426    21-25
11427    26-30
Name: Age, Length: 11428, dtype: category
Categories (11, object): [-20 < 21-25 < 26-

In [222]:
data_2019["YearsCode"]

0         3
1        17
2        35
3         3
4        12
5         4
6        14
7        30
8        40
9        35
10        5
11       10
12       15
13       28
14       22
15       11
16        7
17       20
18       50
19       10
20       16
21        8
22       25
23       16
24       12
25        7
26        8
27        7
28       25
29        6
         ..
11398    12
11399    26
11400     6
11401    35
11402    20
11403     8
11404     1
11405    15
11406    15
11407     5
11408    11
11409    11
11410    15
11411    39
11412     4
11413     8
11414    21
11415    20
11416    17
11417     3
11418    35
11419    26
11420    12
11421     8
11422    14
11423     9
11424    15
11425    15
11426     8
11427    12
Name: YearsCode, Length: 11428, dtype: object

In [178]:
def text_clean(text):
    text = str(text).replace(" ", "_").replace("-", "_").replace(
        ",", "_").replace(".", "").replace("+", "p").replace("#", "s").replace(
            "/", "_").replace("'", "").replace("ʼ", "").replace(
                "(", "_").replace(")", "_").replace("’", "").replace(
                    "__", "_").replace("__", "_").replace("“", "").replace(
                        "”", "").replace(":", "_").replace("&", "_").lower()

    return text

In [179]:
def create_controls(df, exclude):

    controls = {}

    for col in df.columns:
        if col != exclude:
            controls[col] = {"omitted": text_clean(df[col].value_counts().idxmax()), "controls": list(set([x for sub in list(data_2019[col].apply(text_clean).apply(lambda x: str(x).split(";"))) for x in sub]))}
    return controls

In [209]:
data_2019["CodeRevHrs"][data_2019["CodeRevHrs"].apply(type) == float].value_counts().sort_values(ascending=True)

13.00       1
6.50        1
60.00       1
4.50        1
0.38        1
0.40        1
0.30        1
7.19        1
99.00       1
17.00       2
50.00       2
14.00       2
18.00       3
40.00       3
0.01        4
7.50        5
3.50        6
9.00        6
0.25        9
30.00      10
0.10       10
16.00      11
2.50       13
25.00      13
1.50       23
20.00      51
0.00       52
12.00      60
0.50       77
7.00       80
15.00     129
6.00      353
8.00      390
10.00     486
1.00     1106
3.00     1115
4.00     1158
5.00     1611
2.00     1922
Name: CodeRevHrs, dtype: int64

In [180]:
controls = create_controls(data_2019, "Income")

In [183]:
controls

{'Age': {'controls': ['53',
   '33',
   '24',
   '36',
   '63',
   '40',
   '32',
   '18',
   '62',
   '59',
   '29',
   '47',
   '22',
   '23',
   '46',
   '58',
   '99',
   '28',
   '25',
   '72',
   '38',
   '71',
   '69',
   '42',
   '51',
   '39',
   '50',
   '30',
   '48',
   '54',
   '43',
   '67',
   '21',
   '56',
   '68',
   '31',
   '61',
   '52',
   '49',
   '19',
   '20',
   '64',
   '45',
   '65',
   '26',
   '60',
   '66',
   '70',
   '37',
   '27',
   '74',
   '57',
   '41',
   '44',
   '55',
   '34',
   '35'],
  'omitted': '27'},
 'Age1stCode': {'controls': ['14',
   '5',
   '33',
   '24',
   '36',
   '15',
   '40',
   '32',
   '17',
   '18',
   '6',
   '13',
   '10',
   '29',
   '9',
   '22',
   '23',
   '28',
   '25',
   '38',
   '42',
   '51',
   'younger_than_5_years',
   '39',
   '50',
   '30',
   '54',
   '43',
   '21',
   '16',
   '31',
   '11',
   '20',
   '19',
   '12',
   'no_answer',
   '45',
   '26',
   '37',
   '27',
   '41',
   '44',
   '8',
   '7',
   '3

In [ ]:
data_2019 = data_2019[(data_2019["Country"] == "United States") & 
                      (data_2019["Student"] == "No") & 
                      (data_2019["EdLevel"] != "Some college/university study without earning a degree") & 
                      (data_2019["EdLevel"] != "Associate degree") & 
                      (data_2019["Employment"] == "Employed full-time")].copy()

In [ ]:
education_levels_2019 = {"gradHS": "Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)",
                         "gradUni": "Bachelor’s degree (BA, BS, B.Eng., etc.)",
                         "gradDeg": ["Master’s degree (MA, MS, M.Eng., MBA, etc.)",
                         "Other doctoral degree (Ph.D, Ed.D., etc.)",
                         "Professional degree (JD, MD, etc.)"]}

In [ ]:
for k, v in education_levels_2019.items():
    data_2019[k] = data_2019["EdLevel"].map(lambda x: str(x) in v) * 1

In [ ]:
data_2019.rename(mapper={"ConvertedComp": "ln_salary"}, axis=1, inplace=True)
data_2019 = data_2019[data_2019["ln_salary"] > 0].copy()
data_2019["ln_salary"] = np.log(data_2019["ln_salary"])
data_2019.dropna(subset=["ln_salary"], inplace=True)

In [ ]:
data_2019 = data_2019[["ln_salary"] + [key for key in education_levels_2019]].copy()